In [2]:
import osmnx as ox
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic as geopy_distance
from geopy.extra.rate_limiter import RateLimiter
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import networkx as nx
import pandas as pd

In [5]:
import random
# Define a function to generate random coordinates within a given radius of a central point
def generate_random_coordinates(center_lat, center_lon, radius):
    # Generate random distance and angle
    random_distance = random.random() * radius
    random_angle = random.random() * 360
    # Calculate random point coordinates using geopy.distance
    return geopy_distance(kilometers=random_distance).destination((center_lat, center_lon), bearing=random_angle)


# Initialize geolocator with a user-agent
geolocator = Nominatim(user_agent="routing-system-big-data")

# List of areas to geocode
areas = ["Drumul Taberei, Bucharest", "Berceni, Bucharest", "Colentina, Bucharest", "Militari, Bucharest"]
radius = 2  # Radius in kilometers within which to generate house locations
num_locations_per_area = 5  # Number of locations to generate per area

# Get central coordinates of the areas
central_coordinates = {area: geolocator.geocode(area) for area in areas}
# Generate house locations
house_locations = []
for area, location in central_coordinates.items():
    if location:
        for _ in range(num_locations_per_area):
            random_coords = generate_random_coordinates(location.latitude, location.longitude, radius)
            house_locations.append((random_coords[0], random_coords[1]))

# house_locations now contains the generated coordinates for the house locations
house_locations

[(44.41932904790832, 26.03476969022729),
 (44.41464689892644, 26.03323756232944),
 (44.4193943341598, 26.04392705010982),
 (44.420534763967375, 26.031273487266095),
 (44.429325656335386, 26.045533703442068),
 (44.38613648789535, 26.118820092723496),
 (44.40285047392738, 26.131905219466294),
 (44.38926548353081, 26.117532600796366),
 (44.40352043898799, 26.12600025506753),
 (44.39120689152916, 26.125606652642322),
 (44.458873012748704, 26.137478619981657),
 (44.4667288690767, 26.16034142297531),
 (44.47209238163696, 26.161042894853015),
 (44.46625071310259, 26.15390940038925),
 (44.45434470478381, 26.152566860081894),
 (44.436561732930834, 26.010166651746385),
 (44.43019214992797, 26.008694476264942),
 (44.42487293373873, 26.016105059608368),
 (44.434086035331845, 26.03566625620713),
 (44.43426603391605, 26.015105496137785)]

In [6]:
# Constants for the routing problem
CENTER_LOCATION = (44.413162, 26.163739)  # Replace with actual depot coordinates
DIST = 3000  # Adjust as needed
NUM_VEHICLES = 4  # Adjust as needed

# Get the road network graph
G = ox.graph_from_point(CENTER_LOCATION, dist=DIST, network_type='drive')
G = ox.utils_graph.get_largest_component(G, strongly=True)

# Function to calculate distance matrix using networkx
def calculate_distance_matrix(locations, G):
    distance_matrix = []
    for loc1 in locations:
        row = []
        for loc2 in locations:
            node1 = ox.nearest_nodes(G, loc1[1], loc1[0])
            node2 = ox.nearest_nodes(G, loc2[1], loc2[0])
            length = nx.shortest_path_length(G, node1, node2, weight='length')
            row.append(length)
        distance_matrix.append(row)
    return distance_matrix

# Add depot to the list of locations and calculate the distance matrix
all_locations = house_locations + [CENTER_LOCATION]
distance_matrix = calculate_distance_matrix(all_locations, G)

# Create and configure the routing model
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), NUM_VEHICLES, 0)
routing = pywrapcp.RoutingModel(manager)

# Register the transit callback function
transit_callback_index = routing.RegisterTransitCallback(lambda from_index, to_index: distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)])
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint
dimension_name = 'Distance'
routing.AddDimension(transit_callback_index, 0, 30000, True, dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)


In [9]:

# Set the starting and ending nodes for all vehicles at the depot
depot_index = manager.NodeToIndex(0)  # Depot is the first location
# Start and end points for each vehicle
for vehicle_id in range(NUM_VEHICLES):
    start_index = routing.Start(vehicle_id)
    end_index = routing.End(vehicle_id)
    routing.AddDisjunction([start_index], 0)
    routing.AddDisjunction([end_index], 0)
# Solve the routing problem
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

In [ ]:
solution = routing.SolveWithParameters(search_parameters)

In [ ]:

# Output the solution
if solution:
    print("Routes:")
    total_distance = 0
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))
else:
    print("No solution found!")